In [16]:
%load_ext tensorboard
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
from datetime import datetime

%tensorboard --logdir logs/scalars/

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir= "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S"),
    histogram_freq=0,
    write_graph=True,
    write_grads=True
    )

checkpoint_path = "./training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
   checkpoint_path, verbose=1, save_weights_only=True,
   # Save weights, every epoch.
   save_freq='epoch')


n_epochs = 30   # Number of optimization epochs
n_layers = 1    # Number of random layers
n_train = 200    # Size of the train dataset
n_test = 120     # Size of the test dataset
n_batches = 4     # Size of the batches

np.random.seed(0)           # Seed for NumPy random number generator
tf.random.set_seed(0)       # Seed for TensorFlow random number generator



mnist_dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()



# Reduce dataset size
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]

tf.config.get_visible_devices()


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 70313), started 17:38:14 ago. (Use '!kill 70313' to kill it.)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [17]:

# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255

# Add extra dimension for convolution channels
train_images = np.array(train_images[..., tf.newaxis], requires_grad=False)
test_images = np.array(test_images[..., tf.newaxis], requires_grad=False)

n_qubits = 4

dev = qml.device("default.qubit.tf", wires=n_qubits)
# Random circuit parameters
random_weights = np.random.uniform(high=2 * np.pi, size=(n_layers, n_qubits))

@qml.qnode(dev, interface='tf')
def qnode(inputs):
    inputs *= np.pi
    # Encoding of 4 classical input values
    qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')

    # Random quantum circuit
    RandomLayers(random_weights, wires=list(range(n_qubits)))

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(n_qubits)]


In [18]:
class ConvQLayer(qml.qnn.KerasLayer):
    
    def call(self, inputs):
        
        out = tf.Variable(tf.zeros((n_batches, 14, 14, n_qubits)))
        
        # Loop over the coordinates of the top-left pixel of 2X2 squares
        for j in range(0, 28, 2):
            print(j)
            for k in range(0, 28, 2):
                # Process a squared 2x2 region of the image with a quantum circuit
                qnode_inputs = tf.stack([inputs[:, j, k, 0], inputs[:, j, k + 1, 0], inputs[:, j + 1, k, 0], inputs[:, j + 1, k + 1, 0]], axis=1)
                q_results = super().call(qnode_inputs)
                q_results = tf.dtypes.cast(q_results, tf.float32)
                out[:, j // 2, k // 2].assign(q_results)

        return out


qlayer = ConvQLayer(qnode, {}, output_dim=[14, 14, n_qubits])


qlayer.trainable = False

In [19]:


def Q_Model():
    """Initializes and returns a custom Keras model
    which is ready to be trained."""
    model = keras.models.Sequential([
        qlayer,
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")
    ])
    model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

q_model = Q_Model()

q_history = q_model.fit(
    train_images,
    train_labels,
    validation_data=(test_images, test_labels),
    batch_size = n_batches,
    epochs=n_epochs,
    verbose=2, callbacks=[tensorboard_callback, cp_callback],
    shuffle=True
)



Epoch 1/30


0


/opt/conda/envs/QML-QPF/lib/python3.11/site-packages/pennylane/math/utils.py:227: UserWarning: Contains tensors of types {'autograd', 'tensorflow'}; dispatch will prioritize TensorFlow, PyTorch, and  Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


2
4
6
8
10
12
14
16
18
20
22
24
26
0
2
4
6
8
10
12
14
16
18
20
22
24
26
0
2
4
6
8
10
12
14
16
18
20
22
24
26
0
2
4
6
8
10
12
14
16
18
20
22
24
26
0


KeyboardInterrupt: 

In [ ]:
q_model.summary(0)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_q_layer_1 (ConvQLayer  (4, 14, 14, 4)            0         
 )                                                               
                                                                 
 flatten_1 (Flatten)         (4, 784)                  0         
                                                                 
 dense_1 (Dense)             (4, 10)                   7850      
                                                                 
Total params: 7850 (30.66 KB)
Trainable params: 7850 (30.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
